In [245]:
import pandas as pd
import cobra
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import linprog
import scipy
import json

# Community modeling

In this notebook we will implement a method to create community models of two or more species specific metabolic models using cobrapy.

In [246]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.2823653925322476
Growth:  2.558694612613393


In [247]:
for rec in model_SA.reactions:
    rec.lower_bound = max(rec.lower_bound, -1000)
    rec.upper_bound = min(rec.upper_bound, 1000)

In [248]:
snm3 = pd.read_csv("SNM3.csv", sep =";")
snm3.head()

,Compound,BiGG,ModelSeed,KEGG
0,Alanine,ala__L,cpd00035,C00041
1,Arginine,arg__L,cpd00051,C00062
2,Cysteine,cys__L,cpd00084,C00097
3,Glutamic acid,glu__L,cpd00023,C00025
4,Glycine,gly,cpd00033,C00037


In [249]:
BIOMASS_DP = "Growth" 
BIOMASS_SA = "BIOMASS_iYS_wild_type"
models = [model_DP.copy(), model_SA.copy()]

## 1) Defining exchange media

In [250]:
ex_DP_ids = set(map(lambda x:x.id, model_DP.exchanges))
ex_SA_ids = set(map(lambda x:x.id, model_SA.exchanges))
ex_union_ids = ex_DP_ids.union(ex_SA_ids)
print("Number of united exchange reactions: ", len(ex_union_ids))

Number of united exchange reactions:  111


In [251]:
fva_DP = cobra.flux_analysis.flux_variability_analysis(model_DP)
fva_SA = cobra.flux_analysis.flux_variability_analysis(model_SA)                  

In [252]:
# Defining medium with all 111 total exchange reaction
#def compute_exchange(fva_DP, fva_SA):
#    media_exchange_DP = dict()
#    media_exchange_SA = dict()
#    for ex in ex_union_ids:
#        flux = - 10
#        if flux < 0:
#            media_exchange_DP[ex] = abs(flux)
#            media_exchange_SA[ex] = abs(flux)
#        
#    return media_exchange_DP, media_exchange_SA  

In [253]:
def compute_exchange(fva_DP, fva_SA):
    media_exchange_DP = dict()
    media_exchange_SA = dict()
    for ex in ex_union_ids:
        if ex in fva_DP.index and ex in fva_SA.index:
            if not (fva_DP.loc[ex].minimum ==0 and fva_DP.loc[ex].maximum == 0):
                if not (fva_SA.loc[ex].minimum ==0 and fva_SA.loc[ex].maximum == 0):
                    flux = - 10
                    if flux < 0:
                        media_exchange_DP[ex] = abs(flux)
                        media_exchange_SA[ex] = abs(flux)
        elif ex in fva_DP.index:
            # Take just the minimum if not common
            flux = fva_DP.loc[ex].minimum
            if flux < 0:
                media_exchange_DP[ex] = abs(flux)
        elif ex in fva_SA.index:
            # Take just the minimum if not common
            flux = fva_SA.loc[ex].minimum
            if flux < 0:
                media_exchange_SA[ex] = abs(flux)
        else:
            print("error")
    return media_exchange_DP, media_exchange_SA  

In [254]:
media_exchange_DP, media_exchange_SA = compute_exchange(fva_DP,fva_SA)

In [255]:
def save_dict(data, name):
    with open(name, 'w' ) as file:
        json.dump( data, file )

In [256]:
print("Length of media: ", len(media_exchange_DP))
print(media_exchange_DP)
save_dict(media_exchange_DP, "exchange_DP.json")

Length of media:  44
{'EX_ala__L_e': 10.0, 'EX_na1_e': 10, 'EX_lys__L_e': 10, 'EX_phe__L_e': 10, 'EX_pro__L_e': 0.06044704997960771, 'EX_glu__L_e': 10, 'EX_tyr__L_e': 10, 'EX_h_e': 10, 'EX_leu__L_e': 10, 'EX_glyclt_e': 10, 'EX_cu2_e': 0.009257250408242637, 'EX_fe2_e': 10, 'EX_o2_e': 10, 'EX_ile__L_e': 10, 'EX_co2_e': 10, 'EX_arg__L_e': 10, 'EX_cys__L_e': 10, 'EX_thm_e': 10, 'EX_so4_e': 10, 'EX_orn_e': 10, 'EX_nh4_e': 10, 'EX_4abz_e': 0.00018293671908059108, 'EX_ribflv_e': 10, 'EX_gly_e': 10.0, 'EX_mg2_e': 10, 'EX_zn2_e': 10, 'EX_glc__D_e': 10, 'EX_pi_e': 10, 'EX_trp__L_e': 10, 'EX_ca2_e': 0.01467790893862432, 'EX_nac_e': 10, 'EX_fum_e': 10.0, 'EX_thr__L_e': 0.06937020546135564, 'EX_k_e': 10, 'EX_mn2_e': 10, 'EX_val__L_e': 10, 'EX_ni2_e': 0.010022613010017837, 'EX_his__L_e': 10, 'EX_met__L_e': 10, 'EX_h2o_e': 10, 'EX_cobalt2_e': 0.009981830860061667, 'EX_acald_e': 10, 'EX_26dap__M_e': 0.027344801057605284, 'EX_cl_e': 10}


In [257]:
print("Length of media: ", len(media_exchange_SA))
print(media_exchange_SA)
save_dict(media_exchange_SA, "exchange_SA.json")

Length of media:  37
{'EX_na1_e': 10, 'EX_lys__L_e': 10, 'EX_phe__L_e': 10, 'EX_glu__L_e': 10, 'EX_tyr__L_e': 10, 'EX_h_e': 10, 'EX_leu__L_e': 10, 'EX_pyr_e': 10.0, 'EX_glyclt_e': 10, 'EX_fe2_e': 10, 'EX_o2_e': 10, 'EX_urea_e': 10.0, 'EX_ile__L_e': 10, 'EX_co2_e': 10, 'EX_arg__L_e': 10, 'EX_cys__L_e': 10, 'EX_thm_e': 10, 'EX_so4_e': 10, 'EX_orn_e': 10, 'EX_nh4_e': 10, 'EX_ribflv_e': 10, 'EX_ser__L_e': 10.0, 'EX_mg2_e': 10, 'EX_zn2_e': 10, 'EX_glc__D_e': 10, 'EX_pi_e': 10, 'EX_trp__L_e': 10, 'EX_nac_e': 10, 'EX_k_e': 10, 'EX_mn2_e': 10, 'EX_val__L_e': 10, 'EX_his__L_e': 10, 'EX_met__L_e': 10, 'EX_h2o_e': 10, 'EX_acald_e': 10, 'EX_cl_e': 10, 'EX_mobd_e': 1.791086228829375e-05}


In [258]:
import json
exchange_SA = json.loads(open("exchange_SA.json").read())
exchange_DP = json.loads(open("exchange_DP.json").read())

In [259]:
exchange = dict()
for key, val in exchange_SA.items():
    exchange[key] = val 
for key, val in exchange_DP.items():
    exchange[key] = val 

## 1) Constructing of community model explicitely

In [260]:
def create_stoichiometry_matrix(model):
    metabolites = model.metabolites 
    reactions = model.reactions 
    S = np.zeros((len(metabolites), len(reactions)))
    
    met_id = dict()
    rec_id = dict()
    for i,reaction in enumerate(model.reactions):
        rec_id[reaction.id] = i
        for metabolite, stoich in reaction.metabolites.items():
            met_id[metabolite.id] = int(metabolites.index(metabolite))
            S[metabolites.index(metabolite), i] = stoich
    return S, met_id, rec_id 

In [261]:
class Model():
    def __init__(self, model, biomass_function):
        """ This is a new class of metabolic model, capable of flux balance analysis
        Attributes:
        models (list): CobraPy models of single organisms which will be used in construction
        biomass_reactions (list): List of strings containing the ids for the growth reactions
        """
        self.biomass_function = biomass_function
        self.model = model
        self.id = model.id
        # Compute stoichimetry_matrix
        S, met_id, rec_id = create_stoichiometry_matrix(model)
        self.num_reactions = S.shape[1]
        self.num_metabolites = S.shape[0]
        self.stoichiometry_matrix = scipy.sparse.csr_matrix(S)
        self.met_id = met_id
        self.rec_id = rec_id 
        # Set objective
        idx = self.rec_id[biomass_function]
        c = np.zeros(self.num_reactions)
        c[idx] = 1
        self.objective_c = c
        # Set bounds
        self._reset_bounds()
    
    @property
    def reactions(self):
        return self.model.reactions
    @property
    def exchanges(self):
        return self.model.exchanges
    @property
    def metabolites(self):
        return self.model.metabolites
    @property
    def medium(self):
        return self.model.medium

    def set_medium(self, medium):
        ex_ids = [ex.id for ex in self.exchanges]
        new_med = {}
        for key,val in medium.items():
            if key in ex_ids:
                new_med[key] = val
        self.model.medium = new_med
        self._reset_bounds()
        
    def optimize(self, disp=False):
        sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
        sol["fun"] = -sol["fun"] # As we have to minimize
        return sol 
    
    def slim_optimize(self, disp=False):
        sol = self.optimize(disp=disp)
        return sol["fun"]

    def summary(self):
        sol = self.optimize()
        flux = sol["x"]
        ex_ids = [ex.id for ex in self.exchanges]
        fluxes = []
        for ex in ex_ids:
            idx = self.rec_id[ex]
            fluxes.append(flux[idx])
        summary_df = pd.DataFrame({"Exchange reaction": ex_ids, "Flux": fluxes})
        summary_df.sort_values(["Flux"], inplace=True)
        return summary_df

    def _reset_bounds(self):
        self.bounds = []
        for rec in self.model.reactions:
            self.bounds.append((rec.lower_bound, rec.upper_bound))

    def __add__(self, model2):
        """ Adding another model creates a community model """
        return CommunityModel([self,model2], [1.,1.])
    

In [262]:
class CommunityModel(Model):
    def __init__(self, models, weight):
        self.models = models
        self.id = "|".join([model.id for model in models])
        self.shared_exchanges = []
        self.weight = weight
        for model in models:
            for rec in model.exchanges:
                if rec.id not in self.shared_exchanges:
                    self.shared_exchanges.append(rec.id)
        # Create community stoichimetry matrix with shuttel reactions!
        self._shifts = [len(self.shared_exchanges)]
        for i,model in enumerate(models):
            self._shifts.append(self._shifts[i] + model.num_reactions)
        S_EX = -np.eye(self._shifts[0])
        matrices = [S_EX] + [m.stoichiometry_matrix.todense() for m in models]
        S = scipy.linalg.block_diag(*matrices)
        self.num_reactions = S.shape[1]
        self.num_metabolites = S.shape[0]
        for i, id in enumerate(self.shared_exchanges):
            for j,model in enumerate(models):
                if id in model.rec_id:
                    S[i,self._shifts[j] + model.rec_id[id]] = 1
        self.stoichiometry_matrix = scipy.sparse.csr_matrix(S)
        # Cretae objective:
        self._weighted_objective(weight)
        # Create bounds
        self._reset_bounds()
    
    @property
    def reactions(self):
        return [model.reactions for model in self.models]
    @property
    def exchanges(self):
        return self.shared_exchanges
    @property
    def metabolites(self):
        return [model.metabolites for model in self.models]
    @property
    def medium(self):
        medium = {}
        for model in self.models:
            for key,val in model.medium:
                medium[key] = val
        return medium

    def _weighted_objective(self, weight):
        self.weight = weight
        self.objective_c = np.zeros(self._shifts[0])
        for i,model in enumerate(self.models):
            self.objective_c = np.append(self.objective_c, weight[i]*model.objective_c)
    def _reset_bounds(self):
        self.bounds = []
        for id in self.shared_exchanges:
            min_lower_bound = 0
            for model in self.models:
                if id in model.rec_id:
                    rec = model.reactions.get_by_id(id)
                    if rec.lower_bound < min_lower_bound:
                        min_lower_bound = rec.lower_bound 
            self.bounds.append((min_lower_bound, 1000))
        for model in self.models:
            self.bounds += model.bounds

    def set_medium(self, medium):
        for model in self.models:
            model.set_medium(medium)
        self._reset_bounds()

    def summary(self):
        sol = self.optimize()
        flux = sol["x"]
        ex_ids = self.shared_exchanges
        ex_flux = flux[:len(ex_ids)]
        df_ex = pd.DataFrame({"Exchanges": ex_ids, "Flux": ex_flux})
        df_ex.sort_values(["Flux"], inplace=True)
        for i,model in enumerate(self.models):
            shuttel_ids = [ex.id for ex in model.exchanges]
            id = str(model.id) + " Shuttel Flux"
            df_ex[id] = 0.
            for sh in shuttel_ids:
                idx = model.rec_id[sh]
                df_ex[id][ex_ids.index(sh)] = flux[self._shifts[i] +idx]
        
        return df_ex
    
    def set_weights(self, weight):
        self._weighted_objective(weight)
    
    def get_model_growths(self):
        mask = self.objective_c != 0
        sol = self.optimize()
        flux = sol["x"]
        return flux[mask]

In [263]:
model_DP = Model(model_DP, BIOMASS_DP)
print("Growth DP: ",model_DP.slim_optimize())
model_SA = Model(model_SA, BIOMASS_SA)
print("Growth SA: ",model_SA.slim_optimize())

Growth DP:  0.28236539253224774
Growth SA:  2.5586946126118373


<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Unknown solver options: sparse
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})


In [264]:
# Community model
model = model_SA + model_DP
print("Weights 1:1: ", model.slim_optimize())
growths = model.get_model_growths()
print("SA growth: ", growths[0])
print("DP growth: ", growths[1])


Weights 1:1:  2.5586946126136296
SA growth:  2.5586946126136296
DP growth:  -0.0


<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Unknown solver options: sparse
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})


In [265]:
model.set_medium(exchange)

In [266]:
model = model_SA + model_DP
print("Weights 1:1: ", model.slim_optimize())
growths = model.get_model_growths()
print("SA growth: ", growths[0])
print("DP growth: ", growths[1])

Weights 1:1:  2.2982805284808725
SA growth:  2.2982805284808725
DP growth:  -0.0


<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Unknown solver options: sparse
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})


In [267]:
df = model.summary()
df

<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Unknown solver options: sparse
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-262-65ea33eaf871>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ex[id][ex_ids.index(sh)] = flux[self._shifts[i] +idx]
<ipython-input-262-65ea33eaf871>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

,Exchanges,Flux,iYS854 Shuttel Flux,DP_83VPs_KB5 Shuttel Flux
52,EX_pyr_e,-10.000000,-10.000000,0.0
62,EX_urea_e,-10.000000,-10.000000,0.0
12,EX_h_e,-10.000000,-10.000000,-0.0
51,EX_glc__D_e,-10.000000,-10.000000,-0.0
5,EX_acald_e,-10.000000,-10.000000,-0.0
...,...,...,...,...
98,EX_ser__D_e,10.000000,0.000000,10.0
32,EX_etoh_e,11.509173,11.509173,0.0
19,EX_lac__L_e,22.948626,22.948626,0.0
10,EX_h2o_e,23.016521,33.016521,-10.0


In [268]:
df.to_csv("Summary_exchange_shuttels_Exchange_weight_1_1_RM.csv")

In [269]:
model.set_weights([1,10])
print("weights 1:10 ", model.slim_optimize())
growths = model.get_model_growths()
print("SA growth: ", growths[0])
print("DP growth: ", growths[1])

weights 1:10  4.032449748658346
SA growth:  1.2087958233205274
DP growth:  0.28236539253378184


<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Unknown solver options: sparse
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})


In [270]:
df = model.summary()
df

<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-261-5ceee4aecab6>:49: OptimizeWarning: Unknown solver options: sparse
  sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
<ipython-input-262-65ea33eaf871>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ex[id][ex_ids.index(sh)] = flux[self._shifts[i] +idx]
<ipython-input-262-65ea33eaf871>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

,Exchanges,Flux,iYS854 Shuttel Flux,DP_83VPs_KB5 Shuttel Flux
42,EX_o2_e,-10.000000,-9.911929e+00,-0.088071
24,EX_cys__L_e,-10.000000,-1.200710e-01,-9.879929
12,EX_h_e,-10.000000,-1.000000e+01,-0.000000
51,EX_glc__D_e,-10.000000,5.314926e-11,-10.000000
56,EX_ser__L_e,-10.000000,-1.000000e+01,0.000000
...,...,...,...,...
103,EX_h2s_e,9.853386,0.000000e+00,9.853386
32,EX_etoh_e,10.775537,1.077554e+01,0.000000
19,EX_lac__L_e,12.730453,1.273045e+01,0.000000
23,EX_co2_e,26.968431,2.503038e+01,1.938053


In [271]:
df.to_csv("Summary_exchange_shuttels_Exchange_weight_10_1_RM.csv")